<a href="https://colab.research.google.com/github/hanxi898/Polymer-kaggle/blob/main/minidataset_models_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# 1. 环境准备
# ==============================
!pip install rdkit-pypi scikit-learn xgboost joblib deepchem

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import deepchem as dc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 14.7 MB/s eta 0:00:00


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
# ==============================
# 2. 数据读取
# 数据格式要求：csv，至少包含 2 列： SMILES, Tg
# ==============================
df = pd.read_csv("/content/dataset3.csv")  # 例： "SMILES,Tg"
print(df.head())

                                              SMILES          Tg
0  *=Nc1ccc(N=C(C)Nc2ccc(-c3ccc(NC(=*)C)c(C(=O)O)...   89.380459
1   *C(=O)OC(=O)COc1ccc(OCC(=O)OC(=O)c2ccc(*)nc2)cc1  155.970957
2  *C(=O)c1ccc(C(=O)c2ccc(C=C3CCC(=Cc4ccc(*)cc4)C...  192.209684
3  *C=C(*)c1ccc(OCCCCCC(=O)Oc2c(F)c(F)c(F)c(F)c2F...   73.831985
4                     *C=CC1C=CC(*)c2ccc(CCCCCC)cc21    9.704073


In [ ]:
# --- GraphConv 特征 ---
def featurize_graphconv(smiles_list):
    X = []
    try:
        import deepchem as dc
        featurizer = dc.feat.ConvMolFeaturizer()
        for smi in smiles_list:
            try:
                feat = featurizer.featurize([smi])[0]
                arr = feat.get_atom_features().mean(axis=0)
            except:
                arr = np.zeros(75)
            X.append(arr)
    except:
        print("请安装 deepchem 才能使用 GraphConv 特征")
        X = [np.zeros(75)]*len(smiles_list)
    return np.array(X)

# --- Morgan 指纹 ---
def featurize_morgan(smiles_list, radius=2, nBits=2048):
    X = []
    from rdkit import DataStructs
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            arr = np.zeros(nBits)
        else:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
            arr = np.zeros(nBits, dtype=int)
            DataStructs.ConvertToNumpyArray(fp, arr)
        X.append(arr)
    return np.array(X)

# --- 分子描述符 ---
def featurize_desc(smiles_list):
    X = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            arr = np.zeros(len(Descriptors._descList))
        else:
            arr = np.array([d[1](mol) for d in Descriptors._descList], dtype=float)
        X.append(arr)
    return np.array(X)

# --- 混合特征 ---
def featurize_mixed(smiles_list):
    X_morgan = featurize_morgan(smiles_list)
    X_desc = featurize_desc(smiles_list)
    X_graph = featurize_graphconv(smiles_list)
    X_all = np.hstack([X_morgan, X_desc, X_graph])
    return X_all

# ================= 数据准备 =================
X = featurize_mixed(df["SMILES"].tolist())
y = df["Tg"].values

# 替换 NaN/Inf，并归一化
X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerator
[10:46:34] DEPRECATION WARNING: please use MorganGenerat

接下来，尝试不同的深度学习模型（CNN、LSTM/RNN、Transformer)，观察他们的表现

用pytorch，首先需要将数据转为tensor格式

树模型 (RF/XGB) → 接受 [n_samples, n_features]。

深度学习模型 → 通常需要 [n_samples, seq_len, embedding_dim]

所以我们要 reshape 一下特征矩阵

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 转换为 PyTorch Tensor
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

In [ ]:
# ============== 模型定义 ==============
# 1. CNN 模型
class CNNRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(64 * input_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # [batch, 1, features]
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # flatten
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
# 2. LSTM 模型
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # [batch, seq_len=1, features]
        _, (h, _) = self.lstm(x)
        return self.fc(h[-1])


In [ ]:
# 3. Transformer 模型
class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_heads=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim,
                                                   nhead=num_heads,
                                                   batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # [batch, seq_len=1, features]
        x = self.embedding(x)
        x = self.transformer(x)
        return self.fc(x[:, 0, :])  # 取第一个 token


In [ ]:
# ============== 训练函数 ==============
def train_model(model, train_loader, test_loader, epochs=20, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * xb.size(0)
        train_loss = total_loss / len(train_loader.dataset)

        # 验证
        model.eval()
        with torch.no_grad():
            preds, trues = [], []
            for xb, yb in test_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                preds.append(out.cpu())
                trues.append(yb.cpu())
            preds = torch.cat(preds).squeeze()
            trues = torch.cat(trues).squeeze()
            val_loss = criterion(preds, trues).item()

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    return model

In [ ]:
# ============== 选择并训练模型 ==============
input_dim = X_train.shape[1]

print("\n=== CNN ===")
cnn_model = train_model(CNNRegressor(input_dim), train_loader, test_loader)

print("\n=== LSTM ===")
lstm_model = train_model(LSTMRegressor(input_dim), train_loader, test_loader)

print("\n=== Transformer ===")
trans_model = train_model(TransformerRegressor(input_dim), train_loader, test_loader)


=== CNN ===
Epoch 1/20 | Train Loss: 28208.6484 | Val Loss: 23566.0059
Epoch 2/20 | Train Loss: 19178.4141 | Val Loss: 17106.1738
Epoch 3/20 | Train Loss: 11663.1533 | Val Loss: 15902.7715
Epoch 4/20 | Train Loss: 9224.5537 | Val Loss: 20594.5156
Epoch 5/20 | Train Loss: 13058.3691 | Val Loss: 19681.8242
Epoch 6/20 | Train Loss: 12099.4072 | Val Loss: 16663.0820
Epoch 7/20 | Train Loss: 9308.0508 | Val Loss: 14927.0000
Epoch 8/20 | Train Loss: 7845.7793 | Val Loss: 14761.1875
Epoch 9/20 | Train Loss: 7868.1519 | Val Loss: 15160.3232
Epoch 10/20 | Train Loss: 8305.5332 | Val Loss: 15420.0684
Epoch 11/20 | Train Loss: 8443.0586 | Val Loss: 15281.0752
Epoch 12/20 | Train Loss: 8051.6875 | Val Loss: 14773.9043
Epoch 13/20 | Train Loss: 7198.8901 | Val Loss: 14077.7471
Epoch 14/20 | Train Loss: 6108.3994 | Val Loss: 13452.8379
Epoch 15/20 | Train Loss: 5094.7637 | Val Loss: 13165.8535
Epoch 16/20 | Train Loss: 4476.9722 | Val Loss: 13218.4297
Epoch 17/20 | Train Loss: 4303.6162 | Val Loss:

In [ ]:
总体对比：树模型 vs 深度学习模型

In [ ]:
# ============== 树模型部分 ==============
def evaluate_sklearn_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    results = {
        "Train_MSE": mean_squared_error(y_train, y_pred_train),
        "Test_MSE": mean_squared_error(y_test, y_pred_test),
        "Test_MAE": mean_absolute_error(y_test, y_pred_test),
        "Test_R2": r2_score(y_test, y_pred_test),
    }
    return results

# RF
rf_model = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf_results = evaluate_sklearn_model(rf_model, X_train, y_train, X_test, y_test)
print("\n=== Random Forest ===")
print(rf_results)

# XGB
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_results = evaluate_sklearn_model(xgb_model, X_train, y_train, X_test, y_test)
print("\n=== XGBoost ===")
print(xgb_results)


=== Random Forest ===
{'Train_MSE': 523.2371494907771, 'Test_MSE': 4059.7692541384217, 'Test_MAE': 55.23278569759348, 'Test_R2': 0.7324682141117189}

=== XGBoost ===
{'Train_MSE': 7.271185353415391e-06, 'Test_MSE': 3813.0952711721607, 'Test_MAE': 50.431835565761716, 'Test_R2': 0.7487236037814318}


In [ ]:
# ============== 深度学习模型部分 (前面写好的 train_model 可复用) ==============
def evaluate_dl_model(model, train_loader, test_loader, epochs=20):
    model = train_model(model, train_loader, test_loader, epochs=epochs)

    # 最终评估
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    with torch.no_grad():
        preds, trues = [], []
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            preds.append(out.cpu())
            trues.append(yb.cpu())
        preds = torch.cat(preds).squeeze().numpy()
        trues = torch.cat(trues).squeeze().numpy()

    results = {
        "Test_MSE": mean_squared_error(trues, preds),
        "Test_MAE": mean_absolute_error(trues, preds),
        "Test_R2": r2_score(trues, preds),
    }
    return results

input_dim = X_train.shape[1]

print("\n=== CNN ===")
cnn_results = evaluate_dl_model(CNNRegressor(input_dim), train_loader, test_loader, epochs=20)
print(cnn_results)

print("\n=== LSTM ===")
lstm_results = evaluate_dl_model(LSTMRegressor(input_dim), train_loader, test_loader, epochs=20)
print(lstm_results)

print("\n=== Transformer ===")
trans_results = evaluate_dl_model(TransformerRegressor(input_dim), train_loader, test_loader, epochs=20)
print(trans_results)


=== CNN ===
Epoch 1/20 | Train Loss: 28213.3359 | Val Loss: 25434.6328
Epoch 2/20 | Train Loss: 21175.2910 | Val Loss: 19605.1289
Epoch 3/20 | Train Loss: 14462.0488 | Val Loss: 15652.6357
Epoch 4/20 | Train Loss: 9501.7422 | Val Loss: 17317.9570
Epoch 5/20 | Train Loss: 10207.4258 | Val Loss: 20031.8105
Epoch 6/20 | Train Loss: 12521.0771 | Val Loss: 18669.3418
Epoch 7/20 | Train Loss: 11092.2656 | Val Loss: 16296.4326
Epoch 8/20 | Train Loss: 8766.8438 | Val Loss: 14908.9639
Epoch 9/20 | Train Loss: 7463.6147 | Val Loss: 14646.3262
Epoch 10/20 | Train Loss: 7252.2344 | Val Loss: 14909.2441
Epoch 11/20 | Train Loss: 7478.8994 | Val Loss: 15162.1045
Epoch 12/20 | Train Loss: 7588.3540 | Val Loss: 15147.0332
Epoch 13/20 | Train Loss: 7330.9824 | Val Loss: 14819.3154
Epoch 14/20 | Train Loss: 6686.2480 | Val Loss: 14223.9355
Epoch 15/20 | Train Loss: 5733.6992 | Val Loss: 13563.6357
Epoch 16/20 | Train Loss: 4726.7554 | Val Loss: 13057.1768
Epoch 17/20 | Train Loss: 3947.6904 | Val Loss

In [ ]:
# ============== 最终结果汇总 ==============
print("\n📊 模型对比结果")
all_results = {
    "RandomForest": rf_results,
    "XGBoost": xgb_results,
    "CNN": cnn_results,
    "LSTM": lstm_results,
    "Transformer": trans_results
}
for model, res in all_results.items():
    print(f"{model:12s} | Test_MSE: {res['Test_MSE']:.4f} | Test_MAE: {res['Test_MAE']:.4f} | Test_R2: {res['Test_R2']:.4f}")


📊 模型对比结果
RandomForest | Test_MSE: 4059.7693 | Test_MAE: 55.2328 | Test_R2: 0.7325
XGBoost      | Test_MSE: 3813.0953 | Test_MAE: 50.4318 | Test_R2: 0.7487
CNN          | Test_MSE: 11724.2910 | Test_MAE: 99.3856 | Test_R2: 0.2274
LSTM         | Test_MSE: 31632.9492 | Test_MAE: 128.4100 | Test_R2: -1.0846
Transformer  | Test_MSE: 29533.4121 | Test_MAE: 120.2857 | Test_R2: -0.9462


树模型（RF / XGB）明显更优

R² ≈ 0.73–0.75，说明能解释 ~73–75% 的方差。

MAE 在 50 左右，相当不错。

XGBoost 略优于 RF（更低的 MSE/MAE、更高的 R²）。

深度学习模型表现差

CNN 稍微有点学习能力（R²≈0.23），但远不如树模型。

LSTM / Transformer 严重欠拟合（R²<0，表示比“预测均值”还差）。

为什么深度学习效果差？

**数据量问题**：
深度学习需要大量样本（成千上万分子）才能超过树模型；如果你的数据集只有几百/几千个分子，树模型会更稳。

**特征维度问题**：
你现在的输入是 拼接后的手工特征 (Morgan+desc+GraphConv)。这些特征本身就是高层次、稀疏、非序列化的 → 树模型更擅长处理稀疏、离散的输入。
CNN/LSTM/Transformer其实并不适合直接吃这种“全拼接特征矩阵”。

模型设计问题：
目前的 CNN/LSTM/Transformer 都是 简单版本，相当于是“硬套”，没有针对分子数据的特殊结构做优化（不像 Graph Neural Network, ChemBERTa 这种专门为分子设计的模型）。

✅ 建议

**如果数据量不大（< 1w 分子） → 继续用 XGBoost / RF，**并做 超参数调优 (GridSearch/Optuna)。

如果想用深度学习 → 不要用 CNN/LSTM/Transformer 硬套拼接特征，可以尝试：

Graph Neural Networks (**GNN**)：GCN、GIN、MPNN 等，直接吃分子图。

Transformer for SMILES：比如 ChemBERTa, SMILES-BERT。

这样深度模型会比树模型更有优势。

混合方法：

**先用 GNN / Transformer 提取 embedding，再接一个 XGB 做回归，往往比纯 DL 或纯树模型更强。**

**1. 树模型优化**（在已有混合特征上持续优化 XGB/RF ）

目标：通过特征处理 + 超参搜索 + 交叉验证 + 早停，把 XGB/RF 的上限再顶一截

要点

先做简单特征筛选：低方差/高相关剔除（避免噪声与共线）。

用 Optuna 对 XGB 做贝叶斯优化（含早停）；RF 用 RandomizedSearchCV。

用 K 折交叉验证更稳健地估计泛化性能。

In [ ]:
# ========== 路线A：树模型优化 ==========
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import KFold, cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# ---------- 1) 简单特征筛选：低方差 + 相关性裁剪 ----------
def feature_prune(X, corr_thresh=0.97, var_thresh=0.0):
    # 低方差
    vt = VarianceThreshold(threshold=var_thresh)
    X1 = vt.fit_transform(X)
    kept_idx = np.arange(X.shape[1])[vt.get_support()]

    # 高相关裁剪
    corr = np.corrcoef(X1, rowvar=False)
    to_drop = set()
    for i in range(corr.shape[0]):
        if i in to_drop:
            continue
        for j in range(i+1, corr.shape[0]):
            if abs(corr[i, j]) > corr_thresh:
                to_drop.add(j)
    keep2 = [k for t,k in enumerate(kept_idx) if t not in to_drop]
    return X[:, keep2], keep2

X_pruned, kept_cols = feature_prune(X, corr_thresh=0.97, var_thresh=0.0)

# 重新切分（保持你之前的 random_state）
X_tr, X_te, y_tr, y_te = train_test_split(X_pruned, y, test_size=0.2, random_state=42)

# ---------- 2) XGBoost + Optuna（可选：若无optuna则跳过） ----------
try:
    import optuna

    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 2000),
            "max_depth": trial.suggest_int("max_depth", 4, 12),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.2, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 20.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1e-1, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
            "random_state": 42,
            "tree_method": "hist",
        }
        # 5折CV
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = []
        for tr_idx, va_idx in kf.split(X_tr):
            model = XGBRegressor(**params)
            model.fit(
                X_tr[tr_idx], y_tr[tr_idx],
                eval_set=[(X_tr[va_idx], y_tr[va_idx])],
                verbose=False,
                early_stopping_rounds=50
            )
            pred = model.predict(X_tr[va_idx])
            scores.append(-mean_squared_error(y_tr[va_idx], pred))  # 负MSE用于最大化
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=60, show_progress_bar=False)
    best_params = study.best_params
    best_params.update({"random_state": 42, "tree_method": "hist"})
    xgb_best = XGBRegressor(**best_params)
except Exception as e:
    print("未安装 optuna 或运行失败，使用一组较稳健的默认参数。")
    xgb_best = XGBRegressor(
        n_estimators=1200, learning_rate=0.03, max_depth=8,
        subsample=0.9, colsample_bytree=0.9, min_child_weight=5.0,
        reg_alpha=1e-6, reg_lambda=0.5, random_state=42, tree_method="hist"
    )

xgb_best.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=100)
pred = xgb_best.predict(X_te)
print("[A-XGB] MSE/MAE/R2:", mean_squared_error(y_te, pred), mean_absolute_error(y_te, pred), r2_score(y_te, pred))

# ---------- 3) RandomForest 随机搜索 ----------
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
param_dist = {
    "n_estimators": [300, 600, 1000, 1500],
    "max_depth": [None, 12, 16, 24],
    "min_samples_split": [2, 4, 8],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.3, 0.5, 0.8]
}
rs = RandomizedSearchCV(
    rf, param_distributions=param_dist, n_iter=30,
    scoring="neg_mean_squared_error", cv=5, random_state=42, n_jobs=-1, verbose=0
)
rs.fit(X_tr, y_tr)
rf_best = rs.best_estimator_
pred = rf_best.predict(X_te)
print("[A-RF ] MSE/MAE/R2:", mean_squared_error(y_te, pred), mean_absolute_error(y_te, pred), r2_score(y_te, pred))


**2. 深度学习专用模型**（直接吃 SMILES/图）

目标：让模型直接在分子结构上学习，而不是“硬套”到拼接特征。

方案 1：Graph Neural Network（用 DeepChem 的 GraphConv/MPNN）

优点：直接对分子图做消息传递，常见小中数据集表现强。

In [ ]:
# ========== 路线B1：DeepChem GraphConv ==========
import deepchem as dc
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 数据：直接用 SMILES & y
smiles = df["SMILES"].tolist()
y_vals = df["Tg"].values.astype(np.float32)

# 1) featurizer + dataset
featurizer = dc.feat.ConvMolFeaturizer()
X_mols = featurizer.featurize(smiles)
dataset = dc.data.NumpyDataset(X_mols, y_vals)

# 2) 切分
splitter = dc.splits.RandomSplitter()
train_ds, valid_ds, test_ds = splitter.train_valid_test_split(dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1, seed=42)

# 3) 模型：GraphConvModel（回归）
model = dc.models.GraphConvModel(
    n_tasks=1, mode="regression", graph_conv_layers=[128,128],
    dense_layer_size=256, dropout=0.1, learning_rate=1e-3, model_dir="dc_graphconv"
)

# 4) 训练 + 评估
for epoch in range(30):
    loss = model.fit(train_ds, nb_epoch=1, shuffle=True, deterministic=False)
    val_pred = model.predict(valid_ds).ravel()
    val_true = valid_ds.y.ravel()
    val_mse = mean_squared_error(val_true, val_pred)
    print(f"[B1] Epoch {epoch+1:02d} | Val MSE {val_mse:.4f}")

# Test
test_pred = model.predict(test_ds).ravel()
test_true = test_ds.y.ravel()
print("[B1-GraphConv] MSE/MAE/R2:",
      mean_squared_error(test_true, test_pred),
      mean_absolute_error(test_true, test_pred),
      r2_score(test_true, test_pred))


方案 2：ChemBERTa（Transformer 直接吃 SMILES）

优点：用大模型预训练的化学语义（通常对中小数据集非常友好）

In [ ]:
# ========== 路线B2：ChemBERTa 微调 ==========
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datasets import Dataset, DatasetDict

model_name = "seyonec/ChemBERTa-zinc-base-v1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_fn(batch):
    return tokenizer(batch["smiles"], padding="max_length", truncation=True, max_length=128)

# 构造HF Dataset
data = {
    "smiles": df["SMILES"].tolist(),
    "labels": df["Tg"].astype(float).tolist(),
}
ds = Dataset.from_dict(data)
ds = ds.train_test_split(test_size=0.2, seed=42)
ds = DatasetDict({
    "train": ds["train"],
    "test": ds["test"]
})
ds = ds.map(tokenize_fn, batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# 回归头
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1, problem_type="regression")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.reshape(-1)
    labels = labels.reshape(-1)
    mse = mean_squared_error(labels, preds)
    mae = mean_absolute_error(labels, preds)
    r2  = r2_score(labels, preds)
    return {"mse": mse, "mae": mae, "r2": r2}

args = TrainingArguments(
    output_dir="chemberta_reg",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print("[B2-ChemBERTa] ", metrics)


**3. 混合模型**（深度表示 + 强泛化的树模型）

目标：用预训练表示（如 ChemBERTa 的 [CLS] 向量）或 GNN 中间层 embedding，再交给 XGB 回归。通常比“纯DL”更稳。

方案 1：ChemBERTa 向量 + XGBoost

In [ ]:
# ========== 路线C1：ChemBERTa 向量 + XGB ==========
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = "seyonec/ChemBERTa-zinc-base-v1"
tok = AutoTokenizer.from_pretrained(model_name)
enc = AutoModel.from_pretrained(model_name)
enc.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
enc.to(device)

def smiles_embedding(smiles_list, batch_size=64, max_len=128):
    embs = []
    with torch.no_grad():
        for i in range(0, len(smiles_list), batch_size):
            batch = smiles_list[i:i+batch_size]
            toks = tok(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(device)
            out = enc(**toks)  # last hidden states [B, L, H]
            # 取 [CLS] (Roberta系是位置0)
            cls = out.last_hidden_state[:, 0, :].cpu().numpy()
            embs.append(cls)
    return np.vstack(embs)

smiles = df["SMILES"].tolist()
Y = df["Tg"].values.astype(float)
E = smiles_embedding(smiles)  # [N, H]

X_tr, X_te, y_tr, y_te = train_test_split(E, Y, test_size=0.2, random_state=42)

xgb = XGBRegressor(
    n_estimators=1200, learning_rate=0.05, max_depth=8,
    subsample=0.9, colsample_bytree=0.9, random_state=42, tree_method="hist"
)
xgb.fit(X_tr, y_tr, eval_set=[(X_te, y_te)], verbose=False, early_stopping_rounds=100)
pred = xgb.predict(X_te)
print("[C1 CLS+XGB] MSE/MAE/R2:", mean_squared_error(y_te, pred), mean_absolute_error(y_te, pred), r2_score(y_te, pred))


方案 2：GNN 中间层向量 + XGBoost（用 DeepChem 导出 embedding）

DeepChem 的高层 API 不总是直接暴露“倒数第二层向量”。简便做法是：

用 GraphConvModel(..., dense_layer_size=...) 训练好模型；

用 model._model（Keras 模型）取名为 "graph_convolution", "graph_gather" 等中间层输出（如果版本支持）；

或改用 PyTorch Geometric / DGL 自建网络，显式 return embeddings。
下面给一个 可运行的通用版本：训练一个简单 GCN（PyTorch Geometric），抽 embedding 再 XGB。你需要安装 torch-geometric（小坑较多，建议本地/colab装好环境后运行）

In [ ]:
# ========== 路线C2：PyG GCN embedding + XGB ==========
# 仅在已安装 torch_geometric 时使用
try:
    import torch
    from torch_geometric.data import Data
    from torch_geometric.loader import DataLoader
    from torch_geometric.nn import GCNConv, global_mean_pool
    from rdkit import Chem

    # 1) SMILES -> 图数据（极简原子特征：原子序号 one-hot；可自行丰富）
    import numpy as np
    ATOMS = [6, 7, 8, 9, 15, 16, 17, 35, 53]  # 常见元素原子序号，可扩展
    def atom_to_feat(atom):
        z = atom.GetAtomicNum()
        onehot = [1.0 if z==a else 0.0 for a in ATOMS]
        onehot.append(1.0 if z not in ATOMS else 0.0)  # other
        return np.array(onehot, dtype=np.float32)

    def mol_to_pyg(smi):
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            return None
        xs = [atom_to_feat(a) for a in mol.GetAtoms()]
        x = torch.tensor(np.vstack(xs), dtype=torch.float32)
        edges = []
        for b in mol.GetBonds():
            i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
            edges.append([i,j]); edges.append([j,i])
        if len(edges)==0:
            # 单原子分子，构造自环
            edge_index = torch.tensor([[0],[0]], dtype=torch.long)
        else:
            edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
        return Data(x=x, edge_index=edge_index)

    graphs = []
    for smi, tg in zip(df["SMILES"], df["Tg"]):
        g = mol_to_pyg(smi)
        if g is not None:
            g.y = torch.tensor([float(tg)], dtype=torch.float32)
            g.batch_idx = None
            graphs.append(g)

    # 切分
    idx = np.arange(len(graphs))
    rs = np.random.RandomState(42)
    rs.shuffle(idx)
    n_te = int(0.2*len(idx))
    te_idx = idx[:n_te]; tr_idx = idx[n_te:]
    train_graphs = [graphs[i] for i in tr_idx]
    test_graphs  = [graphs[i] for i in te_idx]

    train_loader = DataLoader(train_graphs, batch_size=64, shuffle=True)
    test_loader  = DataLoader(test_graphs, batch_size=128, shuffle=False)

    # 2) 定义GCN
    import torch.nn as nn
    class GCN(nn.Module):
        def __init__(self, in_dim, hid=128, out_dim=1):
            super().__init__()
            self.conv1 = GCNConv(in_dim, hid)
            self.conv2 = GCNConv(hid, hid)
            self.head  = nn.Linear(hid, out_dim)
        def forward(self, data, return_emb=False):
            x, edge_index, batch = data.x, data.edge_index, data.batch
            x = torch.relu(self.conv1(x, edge_index))
            x = torch.relu(self.conv2(x, edge_index))
            emb = global_mean_pool(x, batch)  # [B, hid]
            out = self.head(emb)
            return (out, emb) if return_emb else out

    in_dim = train_graphs[0].x.size(1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = GCN(in_dim).to(device)
    opt = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
    loss_fn = nn.MSELoss()

    # 3) 训练
    for epoch in range(30):
        net.train()
        total = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            opt.zero_grad()
            pred = net(batch)
            loss = loss_fn(pred.squeeze(), batch.y.squeeze())
            loss.backward()
            opt.step()
            total += loss.item() * batch.num_graphs
        print(f"[C2] Epoch {epoch+1:02d} Train Loss {total/len(train_loader.dataset):.4f}")

    # 4) 导出embedding
    net.eval()
    def get_embeddings(loader):
        all_emb, all_y = [], []
        with torch.no_grad():
            for batch in loader:
                batch = batch.to(device)
                _, emb = net(batch, return_emb=True)
                all_emb.append(emb.cpu().numpy())
                all_y.append(batch.y.cpu().numpy())
        return np.vstack(all_emb), np.vstack(all_y).ravel()

    E_tr, y_tr = get_embeddings(train_loader)
    E_te, y_te = get_embeddings(test_loader)

    # 5) XGB 回归
    from xgboost import XGBRegressor
    xgb = XGBRegressor(
        n_estimators=1000, learning_rate=0.05, max_depth=8,
        subsample=0.9, colsample_bytree=0.9, random_state=42, tree_method="hist"
    )
    xgb.fit(E_tr, y_tr, eval_set=[(E_te, y_te)], verbose=False, early_stopping_rounds=100)
    pred = xgb.predict(E_te)
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    print("[C2 GCN-emb+XGB] MSE/MAE/R2:", mean_squared_error(y_te, pred), mean_absolute_error(y_te, pred), r2_score(y_te, pred))

except Exception as e:
    print("PyTorch Geometric 环境不可用，建议改用路线C1或在Colab配置后再试。", e)


**什么时候选哪条路？**

数据量 < 1万分子：优先 路线A（XGB），再试 路线C1（ChemBERTa 向量 + XGB）。

中等数据量 + 计算资源OK：试 路线B2（直接微调 ChemBERTa）。

想融合图结构优势：路线B1 或 路线C2（GNN→XGB）

**在工业界，当数据量较小的时候，深度学习模型效果不理想，而机器学习方法的结果精确的不够的时候，要怎么进行模型建立预测，思路是什么。**

在工业界，数据量较小时，如果你发现

深度学习模型严重欠拟合（比如训练集和验证集都很差）；

传统机器学习模型精度不够；

那思路要从 建模策略 + 特征工程 + 外部信息利用 三个方向去考虑，而不是单纯纠结“深度学习 vs. 机器学习”。

🔑 思路框架
1. 模型层面

小模型优先：深度学习模型在小数据下容易欠拟合或过拟合。此时可以用

树模型（XGBoost、LightGBM、CatBoost）

集成学习（Stacking / Blending / Bagging / Boosting）

简单线性 / 广义线性模型 + 正则化（L1/L2/ElasticNet）

深度学习降维/特征提取：

用预训练模型提特征（NLP: BERT embeddings；CV: ResNet/ViT 特征；化学材料: 分子图神经网络）

再接入传统机器学习模型（SVM/LightGBM）

工业界很多时候是 DL 做 feature extractor + ML 做预测。

2. 特征工程

领域知识嵌入：引入专家手工特征（物理/化学/业务逻辑衍生特征）。

特征交互：自动/半自动构造特征交叉（Polynomial Features、Embedding 交叉）。

降维：在小数据下减少噪声（PCA/Autoencoder 提取低维表征）。

3. 外部信息/数据增强

迁移学习：利用大规模公开数据预训练，再在小数据集上微调。

数据增强：对输入空间做增强（图像增强、SMOTE 过采样、Mixup）。

合成数据：模拟数据/物理建模生成数据（digital twin, GAN）。

4. 建模策略

混合模型：深度学习负责复杂模式捕捉，机器学习负责拟合有限样本。

贝叶斯方法：当数据少时，用贝叶斯建模引入先验，稳定参数估计。

半监督 / 自监督：如果能拿到大量未标注数据，就用自监督学习预训练。

⚙️ 实际工业界 pipeline

先用 ML baseline（如 LightGBM）→ 建立参考指标；

深度学习特征提取（迁移学习）+ ML 再预测；

增强数据 / 加入先验知识；

集成多个模型（ML + DL + 统计模型）。

🎯 核心理解

你说的没错 —— 在小数据集上，深度学习大模型很容易 欠拟合（学不到模式）或过拟合（噪声过多）。
所以工业界更常用 “DL+ML 的混合范式”，而不是硬套纯 DL。
比如：数据量 <1w → ML + 特征工程；数据量 <1M → 迁移学习+ML；数据量大 → DL端到端

**路线总览（Decision Tree）**

数据规模与标签质量先判定

N < 1k 或 标签噪声明显 → 优先 ML + 特征工程，慎用纯 DL。

1k ≤ N < 20k → ML 为主，DL 做特征提取或微调预训练模型。

N ≥ 20k 且标签一致性好 → 可以尝试端到端 DL/GNN；同时保留 ML 作为强基线。

未标注分子充足（>10万）→ 自监督/迁移学习价值大。

化学数据建议优先 Scaffold Split 做验证（而非随机切分），更接近真实泛化难度。

**Phase 0｜准备与评估规范（所有阶段通用）**

数据划分：Scaffold split（train/valid/test ≈ 8/1/1 或 7/2/1）。

指标：回归选 MAE, RMSE, R²；报告均值±标准差（K 折或重复 split）。

质量检查：

label consistency（重复实验样本的方差）

outlier 检查（3σ，或基于 Tanimoto 的孤立点）

任务难度基线：预测全体均值/中位数的 RMSE

化学空间覆盖：用 Tanimoto 距离分布 看 train→test 是否偏移；记录Domain of Applicability (DoA)。

Phase 1｜Baseline（1–3 天搞定）
1A. 特征工程 Baseline（首选）

特征：

Morgan/ECFP（nBits=1024/2048, radius=2/3）

RDKit 描述符（去重/去常量/去高相关）

（可选）简单图嵌入（如 GraphConv 平均池化向量）

模型：LightGBM / XGBoost / RandomForest

技巧：

低方差/高相关裁剪（|ρ|>0.97 剔除）

5/10 折 Scaffold CV

Optuna/RandomSearch 做轻量调参（n_estimators, depth, lr, subsample, colsample）

何时进入下一阶段？

若 R² ≥ 0.6 且业务可接受 → 可直接进入 Phase 4（固化与监控）。

若距离目标还差 (ΔMAE 明显) → 进入 Phase 2（增强与迁移）。

**Phase 2｜增强（特征↑、数据↑、策略↑） **
2A. 特征增强（优先级高）

多源拼接：ECFP + 物化描述符 + 片段计数（BRICS/ErG）

统计降维：PCA/UMAP（仅在维数极高或训练过慢时）

交互特征：对关键理化属性构造二阶交互（避免过多）

DoA 加权：对 test 样本与最近邻 train 的 Tanimoto 相似度做样本权重或“拒绝预测”

2B. 模型增强（仍以 ML 为主）

堆叠/融合：LightGBM + XGB + RF + 线性回归（元学习器用线性/岭回归）

不确定性：

分位数回归（XGB/LightGBM 支持）

Jackknife/袋外误差估计置信区间

GP/NGBoost（小数据时稳）

2C. 数据增强（小数据常见加分项）

SMILES 增广：同一分子多种等价 SMILES（训练时打乱）

对称/同分异构注意：必要时引入手工规则

转移/合成：物理模拟、文献数据合规合并

转向迁移学习/深度学习的时机：

增强后 R² 卡在 0.6–0.75 上不去；

有较多未标注分子、或可获取相似任务的大数据源；

任务对局部子结构/长程相互作用特别敏感，手工特征表达力不足。

**Phase 3｜迁移学习 & 深度学习（在小数据里“用得巧”）**
3A. 预训练 → 微调（推荐顺序）

SMILES 预训练模型（如 ChemBERTa / SMILES-BERT）：

两种用法

冻结底层，仅训练回归头（小数据首选）

逐层解冻微调（样本≥几千再考虑）

输出 embedding → 喂给 XGB/LightGBM（常常最稳）

图神经网络 (GNN)（GIN/MPNN/AttentiveFP）：

先在大规模公开库做自监督（节点/边掩码、对比学习）或使用开源预训练权重

小数据任务上微调或抽 embedding + ML

多任务/迁移：和目标性质物理相关的任务联合训练（LogP、沸点、溶解度…），共享表示

何时用端到端 DL？

N ≥ 20k、标签稳定、且你已验证迁移学习带来显著提升；

有明确证据：模型需要复杂跨原子长程依赖，ML 难以捕捉。

**Phase 4｜工业落地（上线前后都要做）**
4A. 可解释与合规

特征重要度/SHAP：给出对化学家友好的解释（子结构贡献、理化属性贡献）

DoA 报告：对每个预测样本给出“化学空间内/外”的判定与置信度

错误分析：聚类看系统性误差（特定骨架/官能团偏差）

4B. 监控与回流（MLOps）

数据/模型版本化、特征字典与单位管理

线上监控：

输入分布漂移（Tanimoto 到训练集的距离分布）

预测分布、置信度覆盖率

业务 KPI（实验命中率/节省成本）

主动学习循环：

从线上最高不确定性 & 最高潜在收益的分子里抽样做实验

回流增量训练（保持 Scaffold 评估）

4C. 工程化细节

推理延迟：预先缓存指纹/embedding；GPU 仅在批量预测时用

灰度发布：A/B 对比既有规则或旧模型

回退策略：DoA 外样本回退到简单规则或提示“需实验验证”

关键阈值与“何时转向”的经验值（可作团队共识）

N < 1k：不要端到端 DL。ECFP+GBDT 为主；有 ChemBERTa/GNN 预训练就抽 embedding+GBDT。

1k–5k：ML 为主；**迁移学习（冻结）**开始有收益；DL 仅做特征提取。

5k–20k：迁移学习（逐层解冻）与堆叠/融合常带提升；端到端 GNN 可试验。

≥20k：端到端 DL/GNN 与自监督更值得投入，但依然保留 ML 基线与融合。

标签噪声>实验误差的 20–30%：优先提高数据质量/不确定性建模，再谈模型复杂化。

**最小可行实施清单（落地顺序）**

Baseline 周（第 1 周）

ECFP(2048,r=2/3) + RDKit desc → LightGBM/XGB（Scaffold CV）

重要度/SHAP + 错误分析 + DoA 报告

增强与融合 周（第 2–3 周）

特征裁剪 & 交互；SMILES 增广

XGB/LightGBM/RF 堆叠 + 分位数回归输出置信区间

迁移学习 周（第 4–5 周）

ChemBERTa/预训练 GNN 抽 embedding → XGB

验证收益后再试逐层解冻微调

上线准备 周（第 6 周）

监控、DoA、灰度发布、回退策略

主动学习样本选择流程打通（实验回流）

**一句话总结**

小数据分子预测的最佳实践不是“纯 DL 或纯 ML”，而是“ML 强基线 + 化学先验特征 + 预训练表示 + 融合/不确定性 + 主动学习”的组合拳。
只有在数据量和标签质量都足够时，才逐步把比重转向端到端深度学习。